In [1]:
import multiprocessing
from multiprocessing import Pool
from time import sleep

In [2]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              48
On-line CPU(s) list: 0-47
Thread(s) per core:  1
Core(s) per socket:  24
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Gold 6248R CPU @ 3.00GHz
Stepping:            7
CPU MHz:             3002.591
CPU max MHz:         3000.0000
CPU min MHz:         1200.0000
BogoMIPS:            6000.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-23
NUMA node1 CPU(s):   24-47
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 ds_

In [3]:
multiprocessing.cpu_count()

48

In [4]:
# A function that take 1 second to run (prints the process name and sleeps)
def f(i):
    name = multiprocessing.current_process().name
    print(f'{name} is running\n')
    sleep(1)

In [9]:
# Create a Pool of 5 processors and apply the function (runs in 10 seconds)
with Pool(5) as pool:
    for i in range(10):
        pool.apply(f, (i,))

ForkPoolWorker-21 is running

ForkPoolWorker-22 is running

ForkPoolWorker-23 is running

ForkPoolWorker-24 is running

ForkPoolWorker-25 is running

ForkPoolWorker-21 is running

ForkPoolWorker-22 is running

ForkPoolWorker-23 is running

ForkPoolWorker-24 is running

ForkPoolWorker-25 is running



In [8]:
# Instead, we can use map to achieve parallel execution (runs in 2 seconds)
with Pool(5) as pool:
    pool.map(f, range(10))

ForkPoolWorker-16 is running
ForkPoolWorker-17 is running
ForkPoolWorker-18 is running
ForkPoolWorker-19 is running
ForkPoolWorker-20 is running





ForkPoolWorker-17 is running
ForkPoolWorker-18 is running
ForkPoolWorker-19 is running
ForkPoolWorker-16 is running
ForkPoolWorker-20 is running







## Threading

In [12]:
import threading # this is Python's main threading library
from multiprocessing.pool import ThreadPool # this is a pool of theads

In [14]:
# will run in 2 seconds
with ThreadPool(5) as pool:
    pool.map(f, range(10))

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running

MainProcess is running



Did we actually get a speedup with Threads? Not really... the function is only sleeping. Let's consider a function doing actual work

In [35]:
from time import time

def real_work(i):
    sum(range(10000000))

In [43]:
t0 = time()
with ThreadPool(5) as pool:
    pool.map(real_work, range(10))
tf = time()

print(f'{tf-t0:.3f} s')

1.589 s


In [44]:
t0 = time()
with Pool(5) as pool:
    pool.map(real_work, range(10))
tf = time()

print(f'{tf-t0:.3f} s')

0.342 s


Pool is 5 times faster than ThreadPool, so ThreadPool does not actually achieve speedup with more workers.

The TheadPool is limited by Python's Global Interpeter Lock (GIL), whereas Pool uses multiprocessing to achieve a 5X speedup with 5 processors.